# 장식자(Decorator)
- 어떤 함수를 감싸는(Wrapping) 목적의 함수
- [documents](https://wiki.python.org/moin/PythonDecorators)


"데이터 분석할 때에도 장식자 문법을 정확히 아시면, 같은 코드인데 성능을 수십 배 ~ 수천 배 향상시킬 수 있습니다.  
원래 코드를 안 고치고 살짝 Wrapping만 함으로써 성능 향상을 꾀할 수 있습니다."

## 1급 함수
- 함수를 동적으로 생성 가능, 반환값으로 전달 가능
- 함수를 변수처럼 취급 가능

In [1]:
fn1 = lambda x, y: x + y

In [2]:
fn1(1, 2)

3

In [3]:
fn2 = lambda x, y: x + y + 10

In [4]:
fn2(1, 2)

13

In [5]:
base = 10
fn3 = lambda x, y: x + y + base

In [6]:
fn3(1, 2)

13

In [7]:
def mysum1():
    fn = lambda x, y: x + y
    return fn  # 함수를 리턴하는 함수

myfn = mysum1()
myfn(1, 2)

3

In [8]:
mysum1()(1, 2)  # 리턴값 자체가 함수이므로 가능

3

In [9]:
(lambda x, y: x + y)(1, 2)  # lambda 자체가 함수이므로 가능

3

## 예시

In [14]:
def base_10(fn):
    def wrap(x, y):
        return fn(x, y) + 10
    return wrap

def mysum(x, y):
    return x + y

mywrap = base_10(mysum)

In [15]:
mywrap(1, 2)

13

In [16]:
def mymultiply(x, y):
    return x * y

mywrap = base_10(mymultiply)
mywrap(10, 20)

210

### 장식자 문법 적용

In [17]:
def base_10(fn):
    def wrap(x ,y):
        return fn(x, y) + 10
    return wrap

# def mysum(x, y):
#     return x + y

# mysum = base_10(mysum)

@base_10
def mysum(x, y):
    return x + y

In [20]:
mysum(1, 2)

13


## [메모이제이션(Memoization)](https://namu.wiki/w/%EB%A9%94%EB%AA%A8%EC%9D%B4%EC%A0%9C%EC%9D%B4%EC%85%98)
- 동일한 계산을 반복해야 할 경우, 한 번 계산한 결과를 메모리에 저장해 두었다가 꺼내 씀으로써 중복 계산을 방지할 수 있게 하는 기법
- **데이터 분석** 및 **머신 러닝** 등에서도 성능을 높이기 위해 꼭 필요한 매우매우 중요한 기법!
- [Python 구현 예제](http://code.activestate.com/recipes/52201/)

## memoize

In [30]:
import time

cached1 = {}
cached2 = {}


def mylongtimesum(x, y):
    key = (x, y)
    if key not in cached1:
        time.sleep(1)  # 오랜 시간이 걸리는 코드라고 가정
        cached1[key] = x + y + 10
    return cached1[key]


def mylongtimemultiply(x, y):
    key = (x, y)
    if key not in cached2:
        time.sleep(1)  # 오랜 시간이 걸리는 코드라고 가정
        cached2[key] = (x * y) + 10
    return cached2[key]

In [31]:
print(mylongtimesum(1, 2))  # 1초
print(mylongtimesum(1, 2))
print(mylongtimesum(2, 2))  # 1초
print(mylongtimesum(2, 2))
print(mylongtimesum(1, 2))

print(mylongtimemultiply(2, 2))  # 1초
print(mylongtimemultiply(2, 2))
print(mylongtimemultiply(2, 2))

13
13
14
14
13
14
14
14


위 코드에서 두 함수는,
- 동일한 인자에 대한 리턴값이 변하지 않음.(일정)


- 한 번 실행된 함수의 결과를 cached 사전에 저장
  - 인자 => key, 리턴값 => value


- 같은 인자에 대한 함수 재호출 시, 다시 계산할 필요없이 바로 저장된 값을 리턴


- 한 번 실행됐던 호출에 대해서는 계산을 생략하므로, **소요시간 대폭 감소**

### 장식자 출격
- 코드 개선 (중복 제거 및 가독성 up↑)
- 원래 함수에는 주요 로직만 남겨두고, 새로 memoize 함수 구현  


- _"장식자는,_  
_**특정 함수를 wrapping하는 새로운 함수를 만들어서 그 함수를 리턴해주는 것**_"

In [32]:
import time


def memoize(fn):
    cached = {}
    def wrap(x, y):
        key = (x, y)
        if key not in cached:
            cached[key] = fn(x, y)
        return cached[key]
    return wrap


@memoize
def mylongtimesum(x, y):
    time.sleep(1)
    return x + y + 10


@memoize
def mylongtimemultiply(x, y):
    time.sleep(1)
    return (x * y) + 10

In [33]:
print(mylongtimesum(1, 2))  # 1초
print(mylongtimesum(1, 2))
print(mylongtimesum(2, 2))  # 1초
print(mylongtimesum(2, 2))
print(mylongtimesum(1, 2))

print(mylongtimemultiply(2, 2))  # 1초
print(mylongtimemultiply(2, 2))
print(mylongtimemultiply(2, 2))

13
13
14
14
13
14
14
14


## 인자를 받는 장식자

In [39]:
def base_10(fn):
    def wrap(x, y):
        return fn(x, y) + 10
    return wrap

In [40]:
@base_10
def mysum(x, y):
    return x + y

# mysum = base_10(mysum)
mysum(1, 2)

13

- 그럼 base_20, base_30 등은 또 정의해야 되나?

In [43]:
def base(base_i):
    def outer(fn):
        def wrap(x, y):
            return fn(x, y) + base_i
        return wrap
    return outer

In [46]:
base_10 = base(10)

@base_10
def mysum(x, y):
    return x + y

print(mysum(1, 2))
print(mysum(1, 3))

13
14


- 장식자도 함수니까 바로 써주면 더 깔끔하죠잉~

In [47]:
@base(10)
def mysum(x, y):
    return x + y

print(mysum(1, 2))
print(mysum(1, 3))

13
14


In [48]:
@base(20)
def mysum(x, y):
    return x + y

print(mysum(1, 2))
print(mysum(1, 3))

23
24


In [49]:
@base(30)
def mysum(x, y):
    return x + y

print(mysum(1, 2))
print(mysum(1, 3))

33
34


## Quiz) 지정된 조건의 인자만 처리하기
- filter_fn을 통과하지 못하는 인자는 alter_value 값으로 대체하기

In [76]:
# def myfilter(filter_fn, alter_value):
#     def wrap(fn):
#         def inner(*args):
#             numbers = []
#             for i in args:
#                 if filter_fn(i) == False:
#                     i = alter_value
#                 numbers.append(i)
#             return fn(*numbers)
#         return inner
#     return wrap


# def myfilter(filter_fn, alter_value):
#     def wrap(fn):
#         def inner(*args):
#             numbers = [i if filter_fn(i) else alter_value for i in args]
#             return fn(*numbers)
#         return inner
#     return wrap


def myfilter(filter_fn, alter_value):
    def wrap(fn):
        def inner(*args):
            return fn(*[i if filter_fn(i) else alter_value for i in args])
        return inner
    return wrap


@myfilter(lambda i: i%2==0, 0)
def mysum(a, b, c, d, e):
    return a + b + c + d + e


@myfilter(lambda i: i%2==0, 1)
def mymultiply(a, b, c, d, e):
    return a * b * c * d * e


print(mysum(1, 2, 3, 4, 5))  # 6
print(mymultiply(1, 2, 3, 4, 5))  # 8

6
8


## 메모) Quiz 풀다가 알게 된 list comprehension 팁
- else 를 쓸 때는 순서를 바꿔줘야 함

### if 만 쓰는 경우

In [77]:
[i for i in range(1, 6) if i%2 == 0]

[2, 4]

In [78]:
[i if i%2 == 0 for i in range(1, 6)]

SyntaxError: invalid syntax (<ipython-input-78-17880aa2495f>, line 1)

### if ~ else ~ 를 쓰는 경우
- if-else문이 for문 앞에 위치

In [79]:
[i for i in range(1, 6) if i%2 == 0 else 0]

SyntaxError: invalid syntax (<ipython-input-79-93852ccbf283>, line 1)

In [80]:
[i if i%2 == 0 else 0 for i in range(1, 6)]

[0, 2, 0, 4, 0]

- 이유는 잘 모르겠음.
- [List Comprehension 추가 팁](https://code.tutsplus.com/ko/tutorials/list-comprehensions-in-python--cms-26836)